In [1]:
from faulthandler import disable
from unittest import result
from SlicingMachine import TVMSlicer
import tensorflow as tf
import tvm
import tvm.relay as relay
from tvm.contrib import graph_executor 
import numpy as np
import os
import json
import pygraphviz as pgv
from argparse import ArgumentParser
from tvm.relay.build_module import bind_params_by_name
from tvm.relay.dataflow_pattern import *

2022-07-07 18:53:22.456795: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
current_file_path = "./"

np.random.seed(0)
img_size = 512
input_data = np.random.normal(0,1,(1,img_size,img_size,3)).astype(np.float32)
model_keras = tf.keras.models.load_model(current_file_path + '../../tvm-slicer/src/model/{}_{}.h5'.format('unet', img_size))

# tvm result
input_data = input_data.transpose([0, 3, 1, 2])
shape_dict = {"input_1": input_data.shape}
mod, params = relay.frontend.from_keras(model_keras, shape_dict)
target = 'cuda'
dev = tvm.cuda()

# Inserting quantized layer
# upc = UnetPreProcessCallback()
# rewrite(upc, mod['main'])

# uc = UnetCallback(upc.match_node)
# out = rewrite(uc, mod['main'])
# out = relay.Function(out.params, relay.Tuple(uc.tmp + [out.body]), out.ret_type, out.type_params, out.attrs)


2022-07-07 18:53:24.826060: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-07 18:53:24.826154: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-07 18:53:24.850173: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-07 18:53:24.850519: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.665GHz coreCount: 68 deviceMemorySize: 10.75GiB deviceMemoryBandwidth: 573.69GiB/s
2022-07-07 18:53:24.850537: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-07-07 18:53:24.852284: I tensorflow/stream_executor/platfor

In [3]:
class UnetPreProcessCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.var2 = wildcard()
        tuple_node = is_tuple([wildcard(), self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        self.match_node = []
        self.match_node2 = []

    def callback(self, pre, post, node_map):
        var2 = node_map[self.var2][0]
        self.match_node.append(var2)
        self.match_node2.append(pre)
        return post
        
class UnetCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        self.pattern_1 = self.tuple_get_item_node

        self.pattern = self.pattern_1 
        self.match_node = match_node
        self.counter = 0
        self.tmp = []

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(8.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        result_node = relay.annotation.stop_fusion(cast_to_int8)
        self.tmp.append(result_node)
        return result_node

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(8.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern_1.match(pre):
            if pre in self.match_node:
                print("pat 1")
                return self.dequant(self.quant(post))
        return post

class UnetCallback2(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        # self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        # self.pattern_1 = self.tuple_get_item_node
        self.var2 = wildcard()
        tuple_node = is_tuple([wildcard(), self.var2])
        concat_node = is_op('concatenate')(tuple_node)
        self.pattern = concat_node
        # self.pattern = self.pattern_1 
        self.match_node = match_node
        self.counter = 0
        self.tmp = []

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(8.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        result_node = relay.annotation.stop_fusion(cast_to_int8)
        self.tmp.append(result_node)
        return result_node

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(8.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        if self.pattern.match(pre):
            if pre in self.match_node:
                print("pat 1")
                return self.dequant(self.quant(post))
        return post


class UnetMaxPool2dCallback(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        max_pool2d_node = is_op('nn.max_pool2d')(wildcard())
        self.pattern = max_pool2d_node
        self.match_node = []

    def callback(self, pre, post, node_map):
        self.match_node.append(pre)
        return post


class UnetCallback3(DFPatternCallback):
    # A callback class to rewrite the matched pattern to a batch_norm op.
    def __init__(self, match_node, require_type=False):
        super().__init__(require_type)
        super().__init__(rewrite_once=True)

        # self.tuple_get_item_node = is_tuple_get_item(wildcard(), 0)
        # self.pattern_1 = self.tuple_get_item_node
        max_pool2d_node = is_op('nn.max_pool2d')(wildcard())
        self.pattern = max_pool2d_node
        self.match_node = match_node
        self.counter = 0
        self.tmp = []

    def quant(self, node):
        cast_to_int8 = relay.cast(
            relay.clip(
                relay.round(
                    relay.multiply(node, relay.const(8.0))
                ), 
                a_min=-127.0, a_max=127.0
            ),
            dtype="int8"
        )
        result_node = relay.annotation.stop_fusion(cast_to_int8)
        self.tmp.append(result_node)
        return result_node

    def dequant(self, node):
        cast_to_float32 = relay.divide(
            relay.cast(node, dtype='float32'), relay.const(8.0)
        )
        return cast_to_float32

    def callback(self, pre, post, node_map):
        print("match pool2d")

        if self.pattern.match(pre):
            if pre in self.match_node:
                print("pat 1")
                return self.dequant(self.quant(post))
        return post


In [4]:
upc = UnetPreProcessCallback()
rewrite(upc, mod['main'])
uc = UnetCallback(upc.match_node)
out = rewrite(uc, mod['main'])

upc = UnetPreProcessCallback()
rewrite(upc, out)
uc2 = UnetCallback2(upc.match_node2)
out = rewrite(uc2, out)

upc = UnetMaxPool2dCallback()
rewrite(upc, out)
print(len(upc.match_node))
uc2 = UnetCallback3(upc.match_node)
out = rewrite(uc2, out)
print(out)
# out = relay.Function(out.params, relay.Tuple(uc.tmp + [out.body]), out.ret_type, out.type_params, out.attrs)

pat 1
pat 1
pat 1
pat 1
pat 1
pat 1
pat 1
pat 1
4
match pool2d
pat 1
match pool2d
pat 1
match pool2d
pat 1
match pool2d
pat 1
fn (%input_1: Tensor[(1, 3, 512, 512), float32], %v_param_1: Tensor[(16, 3, 3, 3), float32], %v_param_2: Tensor[(16), float32], %v_param_3: Tensor[(16), float32], %v_param_4: Tensor[(16), float32], %v_param_5: Tensor[(16), float32], %v_param_6: Tensor[(16), float32], %v_param_7: Tensor[(16, 16, 3, 3), float32], %v_param_8: Tensor[(16), float32], %v_param_9: Tensor[(16), float32], %v_param_10: Tensor[(16), float32], %v_param_11: Tensor[(16), float32], %v_param_12: Tensor[(16), float32], %v_param_13: Tensor[(32, 16, 3, 3), float32], %v_param_14: Tensor[(32), float32], %v_param_15: Tensor[(32), float32], %v_param_16: Tensor[(32), float32], %v_param_17: Tensor[(32), float32], %v_param_18: Tensor[(32), float32], %v_param_19: Tensor[(32, 32, 3, 3), float32], %v_param_20: Tensor[(32), float32], %v_param_21: Tensor[(32), float32], %v_param_22: Tensor[(32), float32], %v_

In [5]:
len(upc.match_node)


4

In [9]:
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(out, target, params=params)



One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


In [10]:
import pygraphviz as pgv


In [11]:
def show_graph(json_data, file_name=None):
    if type(json_data) == str:
        json_data = json.loads(json_data)
    A = pgv.AGraph(directed=True)
    for node_idx, node in enumerate(json_data['nodes']):
        for src in node['inputs']:
            if 1 == 1:
                src_size = 1
                for i in json_data['attrs']['shape'][1][src[0]]:
                    src_size = src_size * i
                
                dst_size = 1
                for i in json_data['attrs']['shape'][1][node_idx]:
                    dst_size = dst_size * i
                print(src[0], json_data['nodes'][src[0]]['name'], src_size)

                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]) + "[{}]".format(src_size), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]) + "[{}]".format(dst_size))
            else:
                A.add_edge(json_data['nodes'][src[0]]['name'] + '[{}]'.format(src[0]) + '{}'.format(json_data['attrs']['dltype'][1][src[0]]), node['name'] + '[{}]'.format(node_idx) + '{}'.format(json_data['attrs']['dltype'][1][node_idx]))
    if file_name:
        A.draw(file_name + '.png', format='png', prog='dot')


In [12]:
graph_json_raw = lib['get_graph_json']()

show_graph(graph_json_raw, "./test")

0 input_1 786432
1 p0 432
2 p1 16
3 p2 16
4 tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu 4194304
5 p3 2304
6 p4 16
7 p5 16
8 p6 16
9 tvmgen_default_fused_nn_conv2d_add_multiply_add_multiply_round_clip_cast 4194304
10 tvmgen_default_fused_cast_divide 4194304
11 tvmgen_default_fused_nn_max_pool2d_multiply_round_clip_cast 1048576
12 tvmgen_default_fused_cast_divide_1 1048576
13 p7 4608
14 p8 32
15 p9 32
16 tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu_1 2097152
17 p10 9216
18 p11 32
19 p12 32
20 p13 32
21 tvmgen_default_fused_nn_conv2d_add_multiply_add_multiply_round_clip_cast_1 2097152
22 tvmgen_default_fused_cast_divide_2 2097152
23 tvmgen_default_fused_nn_max_pool2d_multiply_round_clip_cast_1 524288
24 tvmgen_default_fused_cast_divide_3 524288
25 p14 18432
26 p15 64
27 p16 64
28 tvmgen_default_fused_nn_conv2d_add_add_nn_leaky_relu_2 1048576
29 p17 36864
30 p18 64
31 p19 64
32 p20 64
33 tvmgen_default_fused_nn_conv2d_add_multiply_add_multiply_round_clip_cast_2 1048576
34 

In [19]:
import json

In [22]:
with open("./src/graph/{}_{}_{}_{}_{}-{}.json".format("unet", "cuda", 512, 3, 0, 123), "w") as json_file:
    json_file.write(json.dumps(json.loads(graph_json_raw)))

In [51]:
tvm_slicer = TVMSlicer(graph_json_raw)
graph_json_front, _, _ = tvm_slicer.slice_graph(0, 91, is_quantize_sliced=True)
graph_json_back, _, _ = tvm_slicer.slice_graph(91 + 1, 119, is_quantize_sliced=True)


##############################
Initial models
model_nodes [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91]
complement_nodes [-1]
##############################
[ 92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109
 110 111 112 113 114 115 116 117 118 119]
######################################
After input analyze
intermediate_nodes []
input_dependency defaultdict(<class 'list'>, {})
model_nodes [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91]
######################################
###############################

In [52]:
show_graph(graph_json_front, "./front")
show_graph(graph_json_back, "./back")


In [53]:
graph_json_back

{'nodes': [{'op': 'null',
   'name': 'input_9',
   'attrs': {'num_outputs': '1',
    'num_inputs': '5',
    'flatten_data': '0',
    'func_name': 'tvmgen_default_fused_nn_conv2d_add_multiply_add_multiply_round_clip_cast',
    'out_layout': '',
    'kernel_layout': 'OIHW',
    'data_layout': 'NCHW',
    'hash': 'c35c41a655f8a1a2'},
   'inputs': []},
  {'op': 'null',
   'name': 'input_91',
   'attrs': {'num_outputs': '1',
    'num_inputs': '2',
    'flatten_data': '0',
    'func_name': 'tvmgen_default_fused_concatenate_multiply_round_clip_cast_2',
    'hash': '9e940466005aa1f3'},
   'inputs': []},
  {'op': 'tvm_op',
   'name': 'tvmgen_default_fused_cast_divide',
   'attrs': {'num_outputs': '1',
    'num_inputs': '1',
    'flatten_data': '0',
    'func_name': 'tvmgen_default_fused_cast_divide',
    'hash': '85165d63e18469f5'},
   'inputs': [[0, 0, 0]]},
  {'op': 'tvm_op',
   'name': 'tvmgen_default_fused_cast_divide_6',
   'attrs': {'num_outputs': '1',
    'num_inputs': '1',
    'flatten_